In [1]:
from audioop import avg
import json
from typing import Callable, Dict, List, Set, Tuple

import numpy as np
from elasticsearch import Elasticsearch
import re
import math
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import HuberRegressor
import csv
from nltk.corpus import stopwords
import re
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from evaluate import load_type_hierarchy, get_type_path
from data_cleaning import preprocess_txt, load_data, load_queries

In [2]:
INDEX_NAME = 'dbpedia_2'
INDEX_SETTINGS = {
    'mappings': {
            'properties': {
                'abstract': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'instance': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                }
            }
        }
    }

In [3]:
es = Elasticsearch()
es.info()

c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'EAS-CZTWRQ3',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'S7F79Gj_R92-RoVRmUCaGw',
 'version': {'number': '7.17.6',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': 'f65e9d338dc1d07b642e14a27f338990148ee5b6',
  'build_date': '2022-08-23T11:08:48.893373482Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
train = load_data('datasets/DBpedia/smarttask_dbpedia_train.json')
#test = load_data('datasets/DBpedia/smarttask_dbpedia_test.json')
test = load_data('results/test_queries_svm_output.json')

In [5]:
type_hierarchy, max_depth = load_type_hierarchy("evaluation\dbpedia\dbpedia_types.tsv")


Loading type hierarchy from evaluation\dbpedia\dbpedia_types.tsv... 761 types loaded (max depth: 7)


In [6]:
training_queries = load_queries(train)
test_queries = load_queries(test)

In [7]:
test_queries

{'dbpedia_7955': {'question': 'what sound pig make french language',
  'category': 'resource',
  'type': ''},
 'dbpedia_22599': {'question': 'where war take place where one commander zuo zongtang',
  'category': 'resource',
  'type': ''},
 'dbpedia_19677': {'question': 'which mountains contained inyo national forest',
  'category': 'resource',
  'type': ''},
 'dbpedia_11163': {'question': 'what seat frankfurter allgemeine zeitung adjacent wetteraukreis',
  'category': 'resource',
  'type': ''},
 'dbpedia_18792': {'question': 'what people buried toronto die',
  'category': 'resource',
  'type': ''},
 'dbpedia_11251': {'question': 'what league major league lacrosse',
  'category': 'resource',
  'type': ''},
 'dbpedia_17993': {'question': 'what series robert guillaume play robert',
  'category': 'resource',
  'type': ''},
 'dbpedia_20546': {'question': 'which political party arunachal pradesh legislative assembly election 2014 leader rishikesh',
  'category': 'resource',
  'type': ''},
 '

In [8]:
def analyze_query(
    es: Elasticsearch, query: str, field: str, index: str = INDEX_NAME
) -> List[str]:
    """Analyzes a query with respect to the relevant index.

    Args:
        es: Elasticsearch object instance.
        query: String of query terms.
        field: The field with respect to which the query is analyzed.
        index: Name of the index with respect to which the query is analyzed.

    Returns:
        A list of query terms that exist in the specified field among the
        documents in the index.
    """
    tokens = es.indices.analyze(index=index, body={"text": query})["tokens"]
    query_terms = []
    for t in sorted(tokens, key=lambda x: x["position"]):
        # Use a boolean query to find at least one document that contains the
        # term.
        hits = (
            es.search(
                index=index,
                query={"match": {field: t["token"]}},
                _source=False,
                size=1,
            )
            .get("hits", {})
            .get("hits", {})
        )
        doc_id = hits[0]["_id"] if len(hits) > 0 else None
        if doc_id is None:
            continue
        query_terms.append(t["token"])
    return query_terms


In [16]:
# GLOB_TERM_DOC_FREQ = {}
# NUM_DOCS = es.count(index=INDEX_NAME)['count']

c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [9]:
def create_query_terms(query_dict: Dict, es: Elasticsearch):
    """Analyze all queries and add terms to dictionary
    Args: 
        query_dict: a dictionary containing {
                                        'query_id': {
                                            'question': 'some question',
                                            'category': 'some category',
                                            'type': 'some type'
                                        }
                                    }
        es: Elasticsearch object instance.
    Returns: 
        A dictionary with query_ids as keys, question, category, 
        type and analized query_terms as values.
    """
    for query_id, query_features in query_dict.items():
        query_dict[query_id]['query_terms'] = analyze_query(es, 
                                            query_features['question'], 
                                            'abstract')
    
    return query_dict

In [12]:
training_queries = create_query_terms(training_queries, es)
test_queries = create_query_terms(test_queries, es)

c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [13]:
def extract_features(query_terms, doc_id, es, index=INDEX_NAME):
    """
    Extracts query features, document features and query-document features of a query and document pair.
    
        Arguments:
            query_terms: List of analyzed query terms.
            doc_id: Document identifier of indexed document.
            es: Elasticsearch object instance.
            index: Name of relevant index on the running Elasticsearch service. 
            
        Returns:
            List of extracted feature values in a fixed order.
    """

    total_docs = es.count(index=INDEX_NAME)['count']
    term_doc_freq_dict = {}
    query = dict(Counter(query_terms))    
    doc_term_freqs = {} 
    tv = es.termvectors(index=index, id=doc_id, fields='abstract', term_statistics=False)    
    for term, term_stat in tv['term_vectors']['abstract']['terms'].items():
        doc_term_freqs[term] = term_stat['term_freq']
    
    idf = []
    for term in query_terms:
        if not term in term_doc_freq_dict:
            n = 0
            hits = es.search(
                index=index, 
                query = {"bool": {"must": {"match": {"abstract": term}}, "must_not": {"match": {"instance": "owl:Thing"}}}}, 
                _source=False, size=1).get('hits',{}).get('hits',{})
            doc_id = (hits[0]['_id'] if (len(hits) > 0) else None)
            if doc_id is not None:
                tv = es.termvectors(index=index, id=doc_id, fields='abstract', term_statistics=True)['term_vectors']['abstract']['terms']
                if term in tv:
                    n = tv[term]['doc_freq']                    
            term_doc_freq_dict[term] = n
            
        n = term_doc_freq_dict[term]
        if n: 
            idf.append(math.log(total_docs/n))

    terms_doc_unique = [v for k,v in doc_term_freqs.items() if k in query] 
    
    return [
        len(query_terms),
        sum(idf),
        max([0] + idf),
        (sum(idf) / max(len(idf),1)),
        sum(doc_term_freqs.values()),
        len(terms_doc_unique),
        sum(terms_doc_unique),
        max([0] + terms_doc_unique),
        (sum(terms_doc_unique) / max(len(query.keys()),1))
    ]

In [66]:
def ltr_feature_vectors(es, training_queries, k=200, index=INDEX_NAME):
    """
    LTR model, generates the feature vectors X and the 
    relevance labels y.
    
    Relevancy scores are defined as:
        0 = Not relevant
        1 = Partially relevant
        2 = Relevant
    
    Arguments:
        es: Elasticsearch object instance.
        k: How many documents to be retrieved from elasticsearch.
        training_queries: training queries.
        index: Name of the index with respect to which the query is analyzed.  
    
    Returns:
        Returns a list of feature vectors, a list of their respective labels (relevance) and a list of 
        instances (type instances) noted from every retrieved doc.
    """
    progress, N = 0, len(training_queries)
    X=[]
    y=[] 
    instances = []
    type_hierarchy, _ = load_type_hierarchy("evaluation\dbpedia\dbpedia_types.tsv")

    for query_id, query_features in training_queries.items():
        type_relevancy = {}        
        for typ in query_features['type']:
            if not typ in type_hierarchy:
                continue
            hierarchy = get_type_path(typ, type_hierarchy)[::-1]
            for v in hierarchy:
                type_relevancy[v] = 1 # Relevant, its in the same hierarchy but in a diff pos.            
                
        for typ in query_features['type']:
            type_relevancy[typ] = 2 # This is the type we want. Give it the highest weight.
            
        if len(type_relevancy) == 0:
            continue

        query = query_features['query_terms']
        if len(query) == 0:
            continue
        hits = es.search(index=index, _source=True, size=k, 
           query = {"bool": {"must": {"match": {"abstract": ' '.join(query)}}, 
           "must_not": {"match": {"instance": "owl:Thing"}}}}
        )['hits']['hits']

        for hit in hits:
            X.append(extract_features(query, hit['_id'], es, index))
            relevancy = 0 # Default = not relevant
            instance_type = hit['_source']['instance']
            if instance_type in type_relevancy:
                relevancy = type_relevancy[instance_type]
            elif instance_type in type_hierarchy:                
                relevancy = get_type_path(instance_type, type_hierarchy)[::-1]
                relevancy = max([(1 if (t in type_relevancy) else 0) for t in relevancy] + [0])
            y.append(relevancy)
            instances.append(instance_type)

        progress += 1
        print(
            "Processing query {}/{} ID {}".format(
                progress, len(training_queries), query_id
            )
        )
        if progress==N:
            break

    return X, y 

In [68]:
def ltr_predict(es, test_queries, model, k=100, index=INDEX_NAME):
    """
    Predicts the category types for test queries. 
    Re-ranks the first k documents from elasticsearch.
    
    Args:
        es: Elasticsearch object instance.
        model: L2R classifier.
        k: How many documents to be retrieved from elasticsearch.
        index: Name of the index with respect to which the query is analyzed.  
    
    Returns:
        A dictionary containing the query_ids and the top 10 predicted types.
    """
    progress, N = 0, len(test_queries)
    results = {}
    for query_id, query_features in test_queries.items():
        query = query_features['query_terms']
        hits = es.search(index=index, _source=True, size=k, 
            query = {"bool": {"must": {"match": {"abstract": ' '.join(query)}}, "must_not": {"match": {"instance": "owl:Thing"}}}}
        )['hits']['hits']
        feat_vecs, instances = [], []
        for hit in hits:
            feat_vecs.append(extract_features(query, hit['_id'], es, index))
            instances.append(hit['_source']['instance'])
        if len(instances) == 0: # No hits!
            results[query_id] = 'N/A'
            continue
        instances_rerank = [instances[idx] for idx in np.argsort(model.predict(feat_vecs))[::-1]]
        results[query_id] = [x[0] for x in Counter(instances_rerank).most_common(10)]

        progress += 1
        print(
            "Processing query {}/{} ID {}".format(
                progress, len(test_queries), query_id
            )
        )

    return results

In [22]:
from sklearn.ensemble import RandomForestClassifier
l2r_model = RandomForestClassifier(n_estimators = 100)
X, y = ltr_feature_vectors(es, training_queries, k=10, index=INDEX_NAME)
l2r_model.fit(X, y)



Loading type hierarchy from evaluation\dbpedia\dbpedia_types.tsv... 761 types loaded (max depth: 7)
Processing query 1/9573 ID dbpedia_14427
Processing query 2/9573 ID dbpedia_3681
Processing query 3/9573 ID dbpedia_12020
Processing query 4/9573 ID dbpedia_10315
Processing query 5/9573 ID dbpedia_1335
Processing query 6/9573 ID dbpedia_6016
Processing query 7/9573 ID dbpedia_3432
Processing query 8/9573 ID dbpedia_16006
Processing query 9/9573 ID dbpedia_278
Processing query 10/9573 ID dbpedia_7661
Processing query 11/9573 ID dbpedia_11196
Processing query 12/9573 ID dbpedia_9060
Processing query 13/9573 ID dbpedia_14631
Processing query 14/9573 ID dbpedia_6019
Processing query 15/9573 ID dbpedia_21638
Processing query 16/9573 ID dbpedia_4600
Processing query 17/9573 ID dbpedia_20525
Processing query 18/9573 ID dbpedia_22649
Processing query 19/9573 ID dbpedia_14877
Processing query 20/9573 ID dbpedia_22706
Processing query 21/9573 ID dbpedia_20349
Processing query 22/9573 ID dbpedia_5

RandomForestClassifier()

In [67]:
from sklearn.linear_model import SGDRegressor
model = SGDRegressor()
X, y = ltr_feature_vectors(es, training_queries, k=10, index=INDEX_NAME)
model.fit(X, y)

Loading type hierarchy from evaluation\dbpedia\dbpedia_types.tsv... 761 types loaded (max depth: 7)
Processing query 1/9573 ID dbpedia_14427
Processing query 2/9573 ID dbpedia_3681
Processing query 3/9573 ID dbpedia_12020
Processing query 4/9573 ID dbpedia_10315
Processing query 5/9573 ID dbpedia_1335
Processing query 6/9573 ID dbpedia_6016
Processing query 7/9573 ID dbpedia_3432
Processing query 8/9573 ID dbpedia_16006
Processing query 9/9573 ID dbpedia_278
Processing query 10/9573 ID dbpedia_7661
Processing query 11/9573 ID dbpedia_11196
Processing query 12/9573 ID dbpedia_9060
Processing query 13/9573 ID dbpedia_14631
Processing query 14/9573 ID dbpedia_6019
Processing query 15/9573 ID dbpedia_21638
Processing query 16/9573 ID dbpedia_4600
Processing query 17/9573 ID dbpedia_20525
Processing query 18/9573 ID dbpedia_22649
Processing query 19/9573 ID dbpedia_14877
Processing query 20/9573 ID dbpedia_22706
Processing query 21/9573 ID dbpedia_20349
Processing query 22/9573 ID dbpedia_5

SGDRegressor()

In [72]:
test_advanced_pointwise = ltr_predict(es, test_queries, model, k=20, index=INDEX_NAME)

c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Processing query 1/2562 ID dbpedia_7955


c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Processing query 2/2562 ID dbpedia_22599


c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


...

Processing query 2558/2562 ID dbpedia_7928
Processing query 2559/2562 ID dbpedia_22949


c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Processing query 2560/2562 ID dbpedia_8479
Processing query 2561/2562 ID dbpedia_4571


c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
c:\Users\bruna\anaconda3\envs\dat640\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Processing query 2562/2562 ID dbpedia_21070


In [73]:
test_advanced_pointwise

{'dbpedia_7955': ['dbo:Language',
  'dbo:ProgrammingLanguage',
  'dbo:AdministrativeRegion',
  'dbo:Island',
  'dbo:Organisation',
  'dbo:Band',
  'dbo:EthnicGroup',
  'dbo:Country',
  'dbo:Food',
  'dbo:Writer'],
 'dbpedia_22599': ['dbo:MilitaryConflict',
  'dbo:MilitaryUnit',
  'dbo:OfficeHolder',
  'dbo:MilitaryPerson',
  'dbo:Royalty',
  'dbo:Settlement',
  'dbo:AdministrativeRegion',
  'dbo:Language',
  'dbo:Sport',
  'dbo:Building'],
 'dbpedia_19677': ['dbo:Settlement',
  'dbo:MountainRange',
  'dbo:Island',
  'dbo:Constellation',
  'dbo:Mountain',
  'dbo:Person',
  'dbo:AdministrativeRegion',
  'dbo:Scientist',
  'dbo:Award',
  'dbo:Place'],
 'dbpedia_11163': ['dbo:City',
  'dbo:PoliticalParty',
  'dbo:Country',
  'dbo:AdministrativeRegion',
  'dbo:Settlement',
  'dbo:Venue',
  'dbo:Language',
  'dbo:Person',
  'dbo:ArtificialSatellite'],
 'dbpedia_18792': ['dbo:EthnicGroup',
  'dbo:Language',
  'dbo:Person',
  'dbo:Monument',
  'dbo:Disease',
  'dbo:Island',
  'dbo:Film',
  'db

In [74]:
def save_test_results(test_res, test):
    all_test_queries = {}
    for x in test:
        all_test_queries.update({x['id']:{
                                    'id': x['id'],
                                    'category': x['category'],
                                    'type': x['type']}
                                })
    
    for key, _ in all_test_queries.items():
        try:
            all_test_queries[key]['type'] = test_res[key]['type']
        except:
            continue

    f = open(f"results/advanced_es_system_output.json", "w")
    json.dump(list(all_test_queries.values()), f)
    f.close()

In [75]:
save_test_results(test_advanced_pointwise, test)

In [76]:
from evaluate import load_ground_truth, load_system_output, evaluate

type_hierarchy, max_depth = load_type_hierarchy('evaluation/dbpedia/dbpedia_types.tsv')
ground_truth = load_ground_truth('datasets/DBpedia/smarttask_dbpedia_test.json', type_hierarchy)
system_output = load_system_output('results/advanced_es_system_output.json')
evaluate(system_output, ground_truth, type_hierarchy, max_depth)

Loading type hierarchy from evaluation/dbpedia/dbpedia_types.tsv... 761 types loaded (max depth: 7)
Loading ground truth from datasets/DBpedia/smarttask_dbpedia_test.json... 
   4369 questions loaded
Loading system predictions from results/advanced_es_system_output.json... 
   4369 predictions loaded


Evaluation results:
-------------------
Category prediction (based on 4369 questions)
  Accuracy: 0.925
Type ranking (based on 4369 questions)
  NDCG@5:  0.383
  NDCG@10: 0.383
